In [ ]:
# 먼저 업로드된 파일을 확인하여 데이터 내용을 살펴보겠습니다.
import pandas as pd

# 파일 경로
file_path = 'csv/집값(평균)_full.csv'

# 파일 로드
data = pd.read_csv(file_path)
data.head()

In [ ]:
# 데이터 전처리: ADD_UP 선택 및 전처리
add_up_data = data[data["AREA"] == "ADD_UP"].iloc[:, 1:]  # "AREA" 제외
add_up_data = add_up_data.T  # 전치
add_up_data.columns = ["House_Price"]  # 열 이름 설정
add_up_data.index = pd.to_datetime(add_up_data.index, format='%Y')  # 연도를 날짜로 변환
# Convert 'House_Price' column to string type before applying .str.replace
add_up_data["House_Price"] = add_up_data["House_Price"].astype(str).str.replace(",", "").astype(float)  # 쉼표 제거 및 숫자로 변환

# 데이터 확인
add_up_data.head()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# ARIMA 모델 학습 및 예측
def arima_forecast(data, order, forecast_steps):
    model = ARIMA(data, order=order)
    result = model.fit()
    forecast = result.get_forecast(steps=forecast_steps)
    forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(years=1), periods=forecast_steps, freq='Y')
    forecast_series = pd.Series(forecast.predicted_mean.values, index=forecast_index)
    return forecast_series, result

# ARIMA 모델 설정 (p=1, d=1, q=1)
order = (1, 1, 1)
forecast_steps = 50  # 향후 50년 예측

# 모델 학습 및 예측
forecast, model_result = arima_forecast(add_up_data["House_Price"], order, forecast_steps)

# 결과 요약
print(model_result.summary())

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(add_up_data, label="Observed")
plt.plot(forecast, label="Forecast", color="orange")
plt.title("House_Price: Observed vs Forecast (ARIMA)")
plt.xlabel("Year")
plt.ylabel("House_Price")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# SEJONG 데이터 추출 및 전처리
sejong_data = data[data["AREA"] == "SEJONG"].iloc[:, 1:]  # "AREA" 제외
sejong_data = sejong_data.T  # 전치
sejong_data.columns = ["House_Price"]  # 열 이름 설정
sejong_data.index = pd.to_datetime(sejong_data.index, format='%Y')  # 연도를 날짜로 변환
# Check if 'House_Price' column is of object (string) type before applying .str.replace
if sejong_data["House_Price"].dtype == object:
    sejong_data["House_Price"] = pd.to_numeric(sejong_data["House_Price"].str.replace(",", ""), errors='coerce')  # 쉼표 제거 및 숫자로 변환
else:
    sejong_data["House_Price"] = pd.to_numeric(sejong_data["House_Price"], errors='coerce')  # If already numeric, just convert

# 2012년 이후 데이터 사용
sejong_data_cleaned = sejong_data.loc["2012":].dropna()

# 데이터 확인
sejong_data_cleaned.head()

In [ ]:
# ARIMA 모델 학습 및 과거 데이터 예측
forecast_steps = 5  # 과거 5년 (2007~2011년) 예측
order = (1, 1, 1)

# 과거 예측을 위한 ARIMA 학습
forecast_backward, model_result = arima_forecast(sejong_data_cleaned["House_Price"], order, forecast_steps)

# 과거 예측 연도 설정
forecast_index = pd.date_range(start="2007-01-01", periods=forecast_steps, freq='Y')
forecast_backward.index = forecast_index

# 예측 결과 확인
forecast_backward

In [ ]:
# SEJONG 지역의 집값 시각화
plt.figure(figsize=(12, 6))
plt.plot(sejong_data_cleaned, label="Observed (2012-2023)", marker="o")
plt.plot(forecast_backward, label="Forecasted (2007-2011)", marker="x", color="orange")
plt.title("SEJONG House Price: Observed vs Forecasted")
plt.xlabel("Year")
plt.ylabel("House_Price")
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression # Import LinearRegression

# Assuming sejong_data_cleaned contains the observed data
X_train = sejong_data_cleaned.index.year.values.reshape(-1, 1)  # Extract years as features
y_train = sejong_data_cleaned["House_Price"].values  # Extract marriage counts as target

# Years to predict (2007-2011)
years_to_predict = [[year] for year in range(2007, 2012)]

# 다항 회귀 모델 학습 (2차 다항식)
degree = 2
poly_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
poly_model.fit(X_train, y_train)

# 2007~2011년 데이터 생성 및 예측
predicted_counts_poly = poly_model.predict(years_to_predict)

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.scatter(X_train, y_train, label="Observed (2012-2023)", color="blue")
plt.plot(years_to_predict, predicted_counts_poly, label=f"Predicted (2007-2011, Degree {degree})", color="orange")
plt.title("SEJONG House Price: Polynomial Regression Prediction (Degree 3)")
plt.xlabel("Year")
plt.ylabel("House Price")
plt.legend()
plt.grid()
plt.show()

# 예측 결과 반환
predicted_counts_poly

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

# 비선형 함수 정의 (예: 지수 함수)
def nonlinear_model(x, a, b, c):
    return a * np.exp(b * (x - 2012)) + c

# 데이터 준비
X_train_nonlinear = sejong_data_cleaned.index.year.values  # Extract year from the index
y_train_nonlinear = sejong_data_cleaned["House_Price"].values

# 비선형 회귀 모델 학습
popt, pcov = curve_fit(nonlinear_model, X_train_nonlinear, y_train_nonlinear, maxfev=10000)

# 2007~2011년 데이터 생성 및 예측
years_to_predict_nonlinear = np.arange(2007, 2012)
predicted_counts_nonlinear = nonlinear_model(years_to_predict_nonlinear, *popt)

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.scatter(X_train_nonlinear, y_train_nonlinear, label="Observed (2012-2023)", color="blue")
plt.plot(years_to_predict_nonlinear, predicted_counts_nonlinear, label="Predicted (2007-2011, Nonlinear)", color="orange")
plt.title("SEJONG House Price: Nonlinear Regression Prediction")
plt.xlabel("Year")
plt.ylabel("House_Price")
plt.legend()
plt.grid()
plt.show()

# 예측 결과 반환
predicted_counts_nonlinear